In [1]:
# all codes come are changed based on LDR code.
import argparse
import torch.optim as optim
import os
import yaml
from mcrgan.default import _C as config
from mcrgan.default import update_config
import torch
from mcrgan.trainer import MCRTrainer
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.models import get_BNN_models
from mcrgan.models import get_noise
from mcrgan.models import DiscriminatorMNIST
from mcrgan.models import weights_init_mnist_model_noise

import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os
from mcrgan.loss import MCRGANloss
import random
random.seed(0)

In [2]:
# data_preprocessing and parameters

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET = 'FMNIST'
ROOT    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 0
DATA_SAMPLE = 50000
LABEL = 0

dataloader, dataset = get_dataloader(
        data_name=DATASET,
        root=ROOT,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)


In [3]:
import collections

def get_netG(netD,netV,net_noise,radius=1):
    models = [netD, netV,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    fed_state_dict = collections.OrderedDict()
    for key in weight_keys:
        fed_state_dict[key] = worker_state_dict[0][key] + radius*worker_state_dict[2][key]*torch.log(1+torch.exp(worker_state_dict[1][key]))
    return fed_state_dict

# new netG already optimized by adam and time learning rate, so we only using netG to update netV
def update_netV(netV,netG,net_noise,fed_state_dict,radius=1):
    models =[netV,netG,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[0][key] - (worker_state_dict[1][key]-fed_state_dict[key])*(worker_state_dict[2][key]*radius)/torch.log(1+torch.exp(-worker_state_dict[0][key]))
    return update_dict
        

In [4]:
#models
# train hyperparameters, batch norm not used var
#store_Var = []
import time
from torch_mimicry.training import scheduler, logger, metric_log

from test_acc import nearsub
import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features2
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os

start_time = time.time()
#global_step = 0
num_steps = 150
n_dis = 1
n_dis2 = 1
numclasses =10
lr1 = 1e-3
lr2 = 1e-3
log_data = metric_log.MetricLog()
data_time = time.time()
iter_dataloader = iter(dataloader)
data, label = next(iter_dataloader)
data_time = time.time() - data_time
real_cpu = data.to(device)
real_label = label.clone().detach()

netD, netG = get_models(DATASET, device)

optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))

Acc_results_D = []
Acc_results_G = []




transform = transforms.Compose(
            [transforms.Resize(32),
             transforms.ToTensor(),
             transforms.Normalize(0.5, 0.5)])

trainset = datasets.FashionMNIST(root=ROOT, train=True, download=True, transform=transform)
testset = datasets.FashionMNIST(root=ROOT, train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(
        testset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)

trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
n_comp = 10


In [5]:
netD, netG

(DataParallel(
   (module): DiscriminatorMNIST(
     (main): Sequential(
       (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
       (1): LeakyReLU(negative_slope=0.2, inplace=True)
       (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
       (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (4): LeakyReLU(negative_slope=0.2, inplace=True)
       (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
       (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (7): LeakyReLU(negative_slope=0.2, inplace=True)
       (8): Conv2d(256, 128, kernel_size=(4, 4), stride=(1, 1), bias=False)
       (9): Flatten(start_dim=1, end_dim=-1)
     )
   )
 ),
 DataParallel(
   (module): GeneratorMNIST(
     (main): Sequential(
       (0): ConvTranspose2d(128, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
       (1)

In [6]:
real_cpu.shape

torch.Size([2048, 1, 32, 32])

In [7]:
netD(real_cpu).shape

torch.Size([2048, 128])

In [8]:
netG(netD(real_cpu)).shape

torch.Size([2048, 1, 32, 32])

In [9]:
netD(netG(netD(real_cpu))).shape

torch.Size([2048, 128])

In [10]:
netD(netG(netD(real_cpu)))

tensor([[ 0.0196,  0.0336, -0.1709,  ..., -0.1897, -0.0953, -0.1602],
        [ 0.0647,  0.0174, -0.1383,  ..., -0.0320, -0.1478, -0.1207],
        [-0.1436,  0.0750, -0.1109,  ..., -0.0521, -0.1125, -0.1259],
        ...,
        [ 0.0117,  0.0464, -0.2375,  ..., -0.1260,  0.0240, -0.0415],
        [ 0.0320, -0.0681, -0.2040,  ..., -0.1116, -0.1577, -0.0863],
        [-0.0462,  0.0264, -0.1784,  ..., -0.1589, -0.1268, -0.1508]],
       device='cuda:0', grad_fn=<DivBackward0>)

In [11]:
# fixed radius case, due to this requires more steps to go to about -200 then set 1500 epoches

n_dis = 1
n_dis2 = 1
mode = 11
gam3 = 1
mcr_gan_loss = MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)

num_steps = 150
lr1 = 1e-3
lr2 = 1e-3
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
meanV_sum = []
radius_list = []
radius = 0

for epoch in range(num_steps):
    for i in range(n_dis):
        netD.zero_grad()
        optD.zero_grad()
        Z = netD(real_cpu)
        fake_img = netG(netD(real_cpu)).detach()
        Z_bar = netD(fake_img)
        errD, errD_EC = mcr_gan_loss(Z, Z_bar, real_label, 1, n_dis)
        print(errD, errD_EC)
        errD.backward()
        optD.step()
        
    for j in range(n_dis2):
        netG.zero_grad()
        optG.zero_grad()
        
        Z = netD(real_cpu)
        fake_img = netG(netD(real_cpu)).detach()
        Z_bar = netD(fake_img)
        
        errG, errD_EC = mcr_gan_loss(Z, Z_bar, real_label, 1, n_dis2)
        print(errG, errD_EC)
        errG = - errG
        errG.backward()
        optG.step()

    
    

tensor(-21.6668, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-8.9963, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.7063, device='cuda:0', grad_fn=<NegBackward0>), tensor(-51.6783, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-13.1055, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-5.0850, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.5383, device='cuda:0', grad_fn=<NegBackward0>), tensor(-31.5278, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-13.1055, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-5.0850, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.5383, device='cuda:0', grad_fn=<NegBackward0>), tensor(-31.5278, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-22.6621, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-8.8750, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.8282, device='cuda:0', grad_fn=<NegBackward0>), tensor(-51.4015, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-22.6621, device='cuda:0', grad_fn=<NegBackward0>) [tenso

tensor(-44.6176, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-22.5431, device='cuda:0', grad_fn=<NegBackward0>), tensor(-17.7092, device='cuda:0', grad_fn=<NegBackward0>), tensor(-77.6653, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-44.4736, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-22.4348, device='cuda:0', grad_fn=<NegBackward0>), tensor(-17.6805, device='cuda:0', grad_fn=<NegBackward0>), tensor(-77.1182, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-44.4736, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-22.4348, device='cuda:0', grad_fn=<NegBackward0>), tensor(-17.6805, device='cuda:0', grad_fn=<NegBackward0>), tensor(-77.1182, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-43.9993, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-22.1160, device='cuda:0', grad_fn=<NegBackward0>), tensor(-17.5371, device='cuda:0', grad_fn=<NegBackward0>), tensor(-76.1872, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-43.9993, device='cuda:0', grad_fn=<NegBackward0>

tensor(-50.8331, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-25.5745, device='cuda:0', grad_fn=<NegBackward0>), tensor(-20.8465, device='cuda:0', grad_fn=<NegBackward0>), tensor(-81.4431, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-50.8331, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-25.5745, device='cuda:0', grad_fn=<NegBackward0>), tensor(-20.8465, device='cuda:0', grad_fn=<NegBackward0>), tensor(-81.4431, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-52.0327, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-26.0821, device='cuda:0', grad_fn=<NegBackward0>), tensor(-21.5297, device='cuda:0', grad_fn=<NegBackward0>), tensor(-82.1684, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-52.0327, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-26.0821, device='cuda:0', grad_fn=<NegBackward0>), tensor(-21.5297, device='cuda:0', grad_fn=<NegBackward0>), tensor(-82.1684, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-52.9918, device='cuda:0', grad_fn=<NegBackward0>

tensor(-58.6221, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-29.0495, device='cuda:0', grad_fn=<NegBackward0>), tensor(-25.0890, device='cuda:0', grad_fn=<NegBackward0>), tensor(-87.5408, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-58.6221, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-29.0495, device='cuda:0', grad_fn=<NegBackward0>), tensor(-25.0890, device='cuda:0', grad_fn=<NegBackward0>), tensor(-87.5408, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-59.9368, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-29.6616, device='cuda:0', grad_fn=<NegBackward0>), tensor(-25.7778, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.7868, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-59.9368, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-29.6616, device='cuda:0', grad_fn=<NegBackward0>), tensor(-25.7778, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.7868, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-60.7192, device='cuda:0', grad_fn=<NegBackward0>

tensor(-62.0000, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-30.5019, device='cuda:0', grad_fn=<NegBackward0>), tensor(-26.9935, device='cuda:0', grad_fn=<NegBackward0>), tensor(-89.4285, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-63.2724, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-31.0700, device='cuda:0', grad_fn=<NegBackward0>), tensor(-27.6866, device='cuda:0', grad_fn=<NegBackward0>), tensor(-90.4587, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-63.2724, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-31.0700, device='cuda:0', grad_fn=<NegBackward0>), tensor(-27.6866, device='cuda:0', grad_fn=<NegBackward0>), tensor(-90.4587, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-63.7187, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-31.3686, device='cuda:0', grad_fn=<NegBackward0>), tensor(-27.8319, device='cuda:0', grad_fn=<NegBackward0>), tensor(-90.6693, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-63.7187, device='cuda:0', grad_fn=<NegBackward0>

tensor(-66.5650, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-32.7239, device='cuda:0', grad_fn=<NegBackward0>), tensor(-29.3049, device='cuda:0', grad_fn=<NegBackward0>), tensor(-92.3360, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-66.5650, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-32.7239, device='cuda:0', grad_fn=<NegBackward0>), tensor(-29.3049, device='cuda:0', grad_fn=<NegBackward0>), tensor(-92.3360, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-66.0279, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-32.5393, device='cuda:0', grad_fn=<NegBackward0>), tensor(-28.9546, device='cuda:0', grad_fn=<NegBackward0>), tensor(-92.1340, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-66.0279, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-32.5393, device='cuda:0', grad_fn=<NegBackward0>), tensor(-28.9546, device='cuda:0', grad_fn=<NegBackward0>), tensor(-92.1340, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-66.0361, device='cuda:0', grad_fn=<NegBackward0>

tensor(-68.8959, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-33.7855, device='cuda:0', grad_fn=<NegBackward0>), tensor(-30.5600, device='cuda:0', grad_fn=<NegBackward0>), tensor(-93.6728, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-69.2581, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-33.9293, device='cuda:0', grad_fn=<NegBackward0>), tensor(-30.7747, device='cuda:0', grad_fn=<NegBackward0>), tensor(-94.0210, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-69.2581, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-33.9293, device='cuda:0', grad_fn=<NegBackward0>), tensor(-30.7747, device='cuda:0', grad_fn=<NegBackward0>), tensor(-94.0210, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-68.9684, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-33.8290, device='cuda:0', grad_fn=<NegBackward0>), tensor(-30.5918, device='cuda:0', grad_fn=<NegBackward0>), tensor(-93.4081, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-68.9684, device='cuda:0', grad_fn=<NegBackward0>

tensor(-71.5918, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-35.0263, device='cuda:0', grad_fn=<NegBackward0>), tensor(-32.0027, device='cuda:0', grad_fn=<NegBackward0>), tensor(-94.8617, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-71.5918, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-35.0263, device='cuda:0', grad_fn=<NegBackward0>), tensor(-32.0027, device='cuda:0', grad_fn=<NegBackward0>), tensor(-94.8617, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-71.4424, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-34.9209, device='cuda:0', grad_fn=<NegBackward0>), tensor(-31.9560, device='cuda:0', grad_fn=<NegBackward0>), tensor(-95.1208, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-71.4424, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-34.9209, device='cuda:0', grad_fn=<NegBackward0>), tensor(-31.9560, device='cuda:0', grad_fn=<NegBackward0>), tensor(-95.1208, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-71.4902, device='cuda:0', grad_fn=<NegBackward0>

tensor(-77.5029, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.3478, device='cuda:0', grad_fn=<NegBackward0>), tensor(-35.5508, device='cuda:0', grad_fn=<NegBackward0>), tensor(-98.9146, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-76.5143, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-36.9578, device='cuda:0', grad_fn=<NegBackward0>), tensor(-34.9588, device='cuda:0', grad_fn=<NegBackward0>), tensor(-98.2535, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-76.5143, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-36.9578, device='cuda:0', grad_fn=<NegBackward0>), tensor(-34.9588, device='cuda:0', grad_fn=<NegBackward0>), tensor(-98.2535, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-76.0296, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-36.9069, device='cuda:0', grad_fn=<NegBackward0>), tensor(-34.5255, device='cuda:0', grad_fn=<NegBackward0>), tensor(-98.2169, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-76.0296, device='cuda:0', grad_fn=<NegBackward0>

In [12]:
#acc at first for NetD
NetD_acc = 0
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features(testloader, netD, netG)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features(testloader, netD, netG)


print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

#acc at first for NetD
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
        
NetD_acc = count/10000
print('NetD, Acc at first', NetD_acc)
Acc_results_D.append(count/10000)

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.01s/it]


----------------------------
Train_z, Test_z
PCA: 0.8412
SVD: 0.8375
NetD, Acc at first 0.8671
